In [1]:
import os
import time
import random
import numpy as np
from math import sqrt
from ElipticCurves import ElipticCurve
from GF import generate_gf_class
from point import generate_point_class

In [2]:
%load_ext autoreload
%autoreload 2

## TEST BASELINE

In [3]:
n = 0x400000000000000000002BEC12BE2262D39BCF14D
d = 0x183F60FDF7951FF47D67193F8D073790C1C9B5A3E
hT = 0x09C9C44277910C9AAEE486883A2EB95B7180166DDF73532EEB76EDAEF52247FF
Ld = 512
Ln = n.bit_length()
m = 163
f = (1 << m) + (1 << 7) + (1 << 6) + (1 << 3) + 1
GF = generate_gf_class(m, f)
A, B = GF(1), GF(0x5FF6108462A2DC8210AB403925E638A19C1455D21)
Point = generate_point_class(GF, A, B)

In [4]:
P = Point(GF(0x72D867F93A93AC27DF9FF01AFFE74885C8C540420), GF(0x0224A9C3947852B97C5599D5F4AB81122ADC3FD9B))

### Signature generation

In [5]:
Q = -P.mul(d)

In [6]:
assert Q == Point(GF(0x057DE7FDE023FF929CB6AC785CE4B79CF64ABDC2DA),GF(0x3E85444324BCF06AD85ABF6AD7B5F34770532B9AA))

In [7]:
h = GF(0x03A2EB95B7180166DDF73532EEB76EDAEF52247FF)

In [8]:
assert GF.to_int(GF(0x09C9C44277910C9AAEE486883A2EB95B7180166DDF73532EEB76EDAEF52247FF), Ln) == 0x03A2EB95B7180166DDF73532EEB76EDAEF52247FF

In [9]:
e = 0x1025E40BD97DB012B7A1D79DE8E12932D247F61C6

In [10]:
eP = P.mul(e)

In [11]:
assert eP==Point(GF(0x42A7D756D70E1C9BA62D2CB43707C35204EF3C67C),GF(0x5310AE5E560464A95DC80286F17EB762EC544B15B))

In [12]:
Fe = eP.x

In [13]:
y = h * Fe

In [14]:
assert y == GF(0x274EA2C0CAA014A0D80A424F59ADE7A93068D08A7)

In [15]:
r = GF.to_int(y, Ln)

In [16]:
s = e + d * r % n

In [17]:
assert s == 0x2100D86957331832B8E8C230F5BD6A332B3615ACA

In [18]:
D = ElipticCurve.concat_to_signature(r, s, Ld)

In [19]:
assert D == 0x000000000000000000000002100D86957331832B8E8C230F5BD6A332B3615ACA00000000000000000000000274EA2C0CAA014A0D80A424F59ADE7A93068D08A7

### Done

In [20]:
old_r, old_s = r, s
old_y = y

### Checking

In [21]:
r, s = ElipticCurve.split_signature(D, Ld)

In [22]:
assert (r, s) == (old_r, old_s)

In [23]:
R = P.mul(s) + Q.mul(r)

In [24]:
assert R == Point(GF(0x42A7D756D70E1C9BA62D2CB43707C35204EF3C67C),GF(0x5310AE5E560464A95DC80286F17EB762EC544B15B))

In [25]:
y = h * R.x

In [26]:
assert y == old_y

In [27]:
r_hat = GF.to_int(y, Ln)

In [28]:
assert r == r_hat

## Test my variant

In [29]:
m = 167
f = 2**167 + 2**6 + 1
A, B = 1, 0x6EE3CEEB230811759F20518A0930F1A4315A827DAC
n = 0x3FFFFFFFFFFFFFFFFFFFFFB12EBCC7D7F29FF7701F
Ln = n.bit_length()
Ld = 512
# assert Ld >= 2 * Ln

In [30]:
curve = ElipticCurve(m, f, A, B, n, Ld)

In [31]:
GF = generate_gf_class(m, f)
GF8 = generate_gf_class(8, (1 << 8) + (1 << 4) + (1 <<3) + (1 << 1) + 1)

### Test GF multiplication

In [32]:
a = GF8(0x53)
b = GF8(0xCA)
assert a * b == GF8(1)

### Test pow

In [33]:
for _ in range(10):
    n1 = GF.random() 
    exp = random.randint(1, 100)
    
    p = n1
    for i in range(exp - 1):
        p *= n1
    assert n1.pow(exp) == p

### Test GF inverse

In [34]:
for _ in range(10):
    number = GF.random() 
    assert number * number.inverse() == GF(1) or number == GF(0)

### Test point on curve

In [35]:
for _ in range(10):
    curve.point_on_curve(curve.random_point())

## Test signing

In [36]:
for _ in range(10):
    print("Good...")
    private_k = curve.gen_private_k()
    public_k = curve.gen_public_k(private_k)
    
    message = os.urandom(10000)
    T, D = curve.sign(message, private_k)
    assert curve.check_signature(T, D, public_k), "bad signature!"

Good...
Good...
Good...
Good...
Good...
Good...
Good...
Good...
Good...
Good...
